# Collect Data by Scraping using Script

In [1]:
# Import packages
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException

## Selenium setup

In [2]:
driver_path = r'C:\Program Files (x86)\chromedriver-win64\chromedriver.exe'

def create_driver():
    options = Options()
    options.add_argument("--start-maximized")  # Start the browser maximized
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver


# Scraping Data from Jobsdb Data

In [ ]:
# Define the base URL and the page URL
base_url = 'https://th.jobsdb.com'
search_data = '/data-jobs'
sec_page_url = '?page=2'

# Fetch the webpage content
page1 = requests.get(urljoin(base_url, search_data))

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(page1.content, 'html.parser')

In [ ]:
box = soup.find('div',{'class':'_1decxdv0 _110qf3s4y _110qf3s4w'})

In [ ]:
company_name = soup.find('a', {'data-automation': 'jobCompany'})
position = soup.find('a', {'data-automation':'jobTitle'})
location = soup.find('a', {'data-automation': 'jobLocation'})
industry = soup.find('a',{'data-automation':'jobClassification'})
date = soup.find('span',{'data-automation':'jobListingDate'})

In [ ]:
company_name.text

In [ ]:
position.text

In [ ]:
location.text

In [ ]:
industry.text

In [ ]:
date.text

In [ ]:
print(company_name.text)
print(position.text)
print(location.text)
print(industry.text)
print(date.text)

In [ ]:
company_name_list = []
position_list = []
location_list = []

## Scrap one page to test

In [ ]:
# Extract job titles
job_title_elements = soup.find_all('a', {'data-automation':'jobTitle'})
job_titles = [div.get_text(strip=True) for div in job_title_elements]

# Extract company names
company_name_elements = soup.find_all('a', {'data-automation': 'jobCompany'})
company_names = [a.get_text(strip=True) for a in company_name_elements]

# Extract Locations
location_elements = soup.find_all('a', {'data-automation': 'jobLocation'})
location = [a.get_text(strip=True) for a in location_elements]

# Extract industry
industry_elements = soup.find_all('a',{'data-automation':'jobClassification'})
industry = [a.get_text(strip=True) for a in industry_elements]

# Extract time
time_elements = soup.find_all('span',{'data-automation':'jobListingDate'})
time = [a.get_text(strip=True) for a in time_elements]

# Combine job titles and company names into a list of tuples
data = list(zip(job_titles, company_names, location, industry,time))

# Create DataFrame
data_jobs = pd.DataFrame(data, columns=['job_title', 'company_name','location','industry','time'])

In [ ]:
data_jobs.head(3)

In [ ]:
all_jobs = []

page_number = 1
previous_content = ""

while True:
    url = f"https://th.jobsdb.com/data-jobs?page={page_number}"
    response = requests.get(url)

    if response.status_code != 200:
        print("Failed to retrieve page")
        break

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract job titles
    job_title_elements = soup.find_all('a', {'data-automation':'jobTitle'})
    job_titles = [div.get_text(strip=True) for div in job_title_elements]

    # Extract company names
    company_name_elements = soup.find_all('a', {'data-automation': 'jobCompany'})
    company_names = [a.get_text(strip=True) for a in company_name_elements]

    # Extract locations
    location_elements = soup.find_all('a', {'data-automation': 'jobLocation'})
    locations = [a.get_text(strip=True) for a in location_elements]

    # Extract industry
    industry_elements = soup.find_all('a',{'data-automation':'jobClassification'})
    industries = [a.get_text(strip=True) for a in industry_elements]

    # Extract time (job posting date)
    time_elements = soup.find_all('span',{'data-automation':'jobListingDate'})
    times = [a.get_text(strip=True) for a in time_elements]

    # Check if extraction was successful
    if len(job_titles) == 0 or len(company_names) == 0 or len(locations) == 0 or len(industries) == 0 or len(times) == 0:
        print(f"No new jobs found on page {page_number}, stopping.")
        break
        
    page_data = list(zip(job_titles, company_names, locations, industries, times))
    all_jobs.extend(page_data)

    # Compare content to determine if the page is the same
    current_content = "".join(job_titles)
    if current_content == previous_content:
        print(f"No new content found on page {page_number}, stopping the loop.")
        break

    # Success Message
    print(f"Successfully scraped page {page_number}")

    previous_content = current_content
    page_number += 1

# Scraped dataframe name 'jobsdb_data'
jobsdb_data = pd.DataFrame(all_jobs, columns=['job_title', 'company_name', 'location', 'industry', 'time'])

In [ ]:
jobsdb_data.shape

In [ ]:
jobsdb_data.isnull().sum()

In [ ]:
jobsdb_data['company_name'].value_counts()

In [ ]:
jobsdb_data['location'].value_counts()

In [ ]:
jobsdb_data['location'].nunique()

In [ ]:
jobsdb_data['location'].unique().tolist()

## Jobtopgun

In [ ]:
jobtopgun_data_1st = 'https://www.jobtopgun.com/en/jobs?keywords=data'
jobtopgun_data_morepages = '&page=2'
jobtopgun_data_secondp = jobtopgun_data_1st + jobtopgun_data_morepages

In [ ]:
driver_jobtopgun = webdriver.Chrome()  
jobtopgun_jobs_data = []

# Starting page number
page_number = 1

while True:
    # Construct URL with the current page number
    url = f'https://www.jobtopgun.com/en/jobs?keywords=data&page={page_number}'
    print(f"Navigating to jobtopgun 'data' search page number: {page_number}")
    
    try:
        # Navigate to the webpage
        driver_jobtopgun.get(url)
        wait = WebDriverWait(driver_jobtopgun, 10)
        original_window = driver_jobtopgun.current_window_handle  # Hold Current windows

        jobtopgun_data_firstpage = []
        i = 1  # Start with the first job listing

        while True:
            xpath = f'//*[@id="scrollable-job-cards-container"]/a[{i}]/div[1]'
            try:
                # Wait for the element to be clickable
                element = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
                element.click()

                # Wait for the new window/tab to open and switch to it
                wait.until(EC.number_of_windows_to_be(2))
                new_window = [window for window in driver_jobtopgun.window_handles if window != original_window][0]
                driver_jobtopgun.switch_to.window(new_window)

                # Scrape data from the job detail page
                try:
                    # Details to scrape data
                    job_title = driver_jobtopgun.find_element(By.XPATH, '//h1[@class="font-medium text-sub-primary text-lg"]').text
                    company_name = driver_jobtopgun.find_element(By.XPATH, '//span[@class="flex-1 font-medium text-lg"]').text
                    industry = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[1]/div/div[1]/div[1]/section/div[1]/span').text
                    posted_time = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[1]/div/div[1]/div[1]/section/div/span').text
                    experience = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[1]/div/div[2]/span').text
                    salary = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[1]/div/div[2]/span').text
                    education = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[1]/div/div[4]/span').text
                    location = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[1]/div/div[5]/span').text
                    responsibility = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[2]/div').text
                    requirements = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[3]').text
                    welfare_and_benefits = driver_jobtopgun.find_element(By.XPATH, '//*[@id="job-details"]/div[2]/section[4]').text
                    
                    job_info = {
                        "job_title": job_title,
                        "company_name": company_name,
                        "industry": industry,
                        "job_url": driver_jobtopgun.current_url,
                        "posted_time": posted_time,
                        "experience": experience,
                        "salary": salary,
                        "education": education,
                        "location": location,
                        "responsibility": responsibility,
                        "requirements": requirements,
                        "welfare_and_benefits": welfare_and_benefits
                    }
                    jobtopgun_data_firstpage.append(job_info)
                    print(f"Scraped data for job [{i}]: {job_info}")
                except Exception as e:
                    print(f"Failed to scrape data for job [{i}]: {e}")
                
                driver_jobtopgun.close()
                driver_jobtopgun.switch_to.window(original_window)

            except Exception as e:
                print(f"Failed to handle element a[{i}]: {e}")
                break  # Exit the loop if element is not found or not clickable

            i += 1  # Move to the next job listing

        # If no jobs are found, assume there are no more pages to scrape
        if not jobtopgun_data_firstpage:
            print(f"No data found on page {page_number}. Stopping.")
            break

        # Append the current page's data to the overall data list
        jobtopgun_jobs_data.extend(jobtopgun_data_firstpage)

        # Increment page number to move to the next page
        page_number += 1

    except Exception as main_exception:
        print(f"Error while navigating to page {page_number}: {main_exception}")
        break

driver_jobtopgun.quit()

print(f"Scraping complete. Total jobs scraped: {len(jobtopgun_jobs_data)}")


In [ ]:
jobtopgun_jobs_data = pd.DataFrame(jobtopgun_jobs_data)

In [ ]:
jobtopgun_jobs_data

## Job Bkk

In [3]:
search_word = 'data'

In [4]:
driver_jobbkk = webdriver.Chrome()  
jobbkk_jobs_data = []

In [5]:
# WebDriver
driver_jobbkk = webdriver.Chrome()
wait = WebDriverWait(driver_jobbkk, 10)  

try:
    # Open the main URL
    driver_jobbkk.get("https://www.jobbkk.com/jobs/lists/1/%E0%B8%AB%E0%B8%B2%E0%B8%87%E0%B8%B2%E0%B8%99,data,%E0%B8%97%E0%B8%B8%E0%B8%81%E0%B8%88%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A7%E0%B8%B1%E0%B8%94,%E0%B8%97%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A1%E0%B8%94.html?keyword_type=3&member_user_id=1")

    # Click the element to open a new tab
    element_to_open_new_tab = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/section[7]/article/section/div[1]/div[6]/div/div[3]/div/ul/li[3]/a')))
    driver_jobbkk.execute_script("window.open(arguments[0].href);", element_to_open_new_tab)

    # Switch to the new tab
    WebDriverWait(driver_jobbkk, 10).until(lambda d: len(d.window_handles) == 2)  # Wait for the new tab to open
    original_window = driver_jobbkk.current_window_handle
    new_window = [window for window in driver_jobbkk.window_handles if window != original_window][0]
    driver_jobbkk.switch_to.window(new_window)
    
    # Get page source and parse it with BeautifulSoup
    page_source = driver_jobbkk.page_source
    soup_bkk = BeautifulSoup(page_source, 'html.parser')

    # Scrape data from the new tab
    try:
        job_title = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[2]/p').text
        company_name = soup_bkk.find('p', class_='textRed fontSubHead font-DB-HeaventRounded-Bold').text
        industry = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[1]/div/div/p[2]').text 
        salary = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[7]').text
        fulltime = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[4]').text
        location = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[1]').text
        work_day = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[8]').text
        dayoff = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[9]').text
        workhour = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[10]').text
        responsibility = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[4]/div').text
        gender = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[1]').text
        age = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[2]').text
        education = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[3]').text
        experience = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[4]').text
        job_requirement = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[6]/div').text
        job_requirement2 = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[7]/div').text
        welfare_and_benefits = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[8]/div').text
        welfare_and_benefits2 = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[9]/div').text
        company_contact = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[10]/div').text
        job_url = driver_jobbkk.current_url
        
        jobbkk_info = {
            'job_title': job_title,
            'company_name': company_name,
            'industry': industry,
            'salary': salary,
            'fulltime': fulltime,
            'location': location,
            'workday': work_day,
            'dayoff': dayoff,
            'workhour': workhour,
            'responsibility': responsibility,
            'gender': gender,
            'age': age,
            'education': education,
            'experience': experience,
            'job_requirement': job_requirement,
            'job_requirement2': job_requirement2,
            'welfare_and_benefits': welfare_and_benefits,
            'welfare_and_benefits2': welfare_and_benefits2,
            'company_contact': company_contact,
            'job_url': job_url
        }
        print(jobbkk_info)
        
    except Exception as e:
        print(f"Error scraping data: {e}")

    # Close the new tab and switch back 
    driver_jobbkk.close()
    driver_jobbkk.switch_to.window(original_window)

finally:
    print('Done scraping')
    driver_jobbkk.quit()


{'job_title': 'Data Engineer รับสมัครด่วน', 'company_name': 'บริษัท ซอฟท์นิกซ์ เทคโนโลยี จำกัด', 'industry': 'ประเภทธุรกิจ : คอมพิวเตอร์-ไอที', 'salary': 'เงินเดือน(บาท) : 25,000 - 40,000', 'fulltime': 'รูปแบบงาน : งานประจำ', 'location': 'สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตดินแดง)', 'workday': 'วันทำงาน : จันทร์-ศุกร์', 'dayoff': 'วันหยุด : วันเสาร์, วันอาทิตย์', 'workhour': 'เวลาทำงาน : 09:00 - 18:00', 'responsibility': 'รับผิดชอบงาน Data Integration, Data Pipeline Management\nออกแบบ Data Architecture ติดตั้ง Big Data Platform และเครื่องมือที่เกี่ยวข้อง\nออกแบบโครงสร้างการจัดเก็บและแนวทางการจัดเก็บข้อมูลบน Data Platform\nศึกษาและค้นคว้าเทคโนโลยีที่เหมาะสมเพื่อพัฒนาสินค้าของเราให้มีคุณภาพที่ดีขึ้น และพัฒนาผลิตภัณฑ์ให้เป็นไปตามเป้าหมายของบริษัท', 'gender': 'เพศ : ไม่ระบุ', 'age': 'อายุ(ปี) : 26 - 35', 'education': 'ระดับการศึกษา : ปริญญาตรี ขึ้นไป', 'experience': 'ประสบการณ์(ปี) : 3 - 5', 'job_requirement': 'มีประสบการณ์ด้านการทำงาน Data Warehouse, ETL\nเชี่ยวชาญ SQL, NoSQL และเขียน P

In [ ]:
jobbkk_info

In [8]:

driver_jobbkk = webdriver.Chrome()
wait = WebDriverWait(driver_jobbkk, 10)  # Adjust the timeout as needed

all_jobs = []
try:
    # Open the main URL
    driver_jobbkk.get("https://www.jobbkk.com/jobs/lists/1/%E0%B8%AB%E0%B8%B2%E0%B8%87%E0%B8%B2%E0%B8%99,data,%E0%B8%97%E0%B8%B8%E0%B8%81%E0%B8%88%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A7%E0%B8%B1%E0%B8%94,%E0%B8%97%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A1%E0%B8%94.html?keyword_type=3&member_user_id=1")
    
    # Loop through elements starting from index 6, with a step of 2
    for x in range(6, 20, 2):
        try:
            xpath = f'/html/body/section[7]/article/section/div[1]/div[{x}]/div/div[3]/div/ul/li[3]/a'
            element_to_open_new_tab = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            driver_jobbkk.execute_script("window.open(arguments[0].href);", element_to_open_new_tab)

            # Switch to the new tab
            WebDriverWait(driver_jobbkk, 10).until(lambda d: len(d.window_handles) == 2)  # Wait for the new tab to open
            original_window = driver_jobbkk.current_window_handle
            new_window = [window for window in driver_jobbkk.window_handles if window != original_window][0]
            driver_jobbkk.switch_to.window(new_window)

            page_source = driver_jobbkk.page_source
            soup_bkk = BeautifulSoup(page_source, 'html.parser')

            # Scrape data from the new tab
            try:
                job_title = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[2]/p').text
                company_name = soup_bkk.find('p', class_='textRed fontSubHead font-DB-HeaventRounded-Bold').text
                industry = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[1]/div/div/p[2]').text 
                salary = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[7]').text
                fulltime = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[4]').text
                location = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[1]').text
                work_day = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[8]').text
                dayoff = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[9]').text
                workhour = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/div[3]/p[10]').text
                responsibility = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[4]/div').text
                gender = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[1]').text
                age = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[2]').text
                education = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[3]').text
                experience = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[5]/div/p[4]').text
                job_requirement = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[6]/div').text
                job_requirement2 = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[7]/div').text
                welfare_and_benefits = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[8]/div').text
                welfare_and_benefits2 = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[9]/div').text
                company_contact = driver_jobbkk.find_element(By.XPATH, '/html/body/section[5]/article/div/article/section[2]/article[1]/div/section[10]/div').text
                job_url = driver_jobbkk.current_url
                
                jobbkk_info = {
                    'job_title': job_title,
                    'company_name': company_name,
                    'industry': industry,
                    'salary': salary,
                    'fulltime': fulltime,
                    'location': location,
                    'workday': work_day,
                    'dayoff': dayoff,
                    'workhour': workhour,
                    'responsibility': responsibility,
                    'gender': gender,
                    'age': age,
                    'education': education,
                    'experience': experience,
                    'job_requirement': job_requirement,
                    'job_requirement2': job_requirement2,
                    'welfare_and_benefits': welfare_and_benefits,
                    'welfare_and_benefits2': welfare_and_benefits2,
                    'company_contact': company_contact,
                    'job_url': job_url
                }
                print(jobbkk_info)
                all_jobs.append(jobbkk_info)
            except Exception as e:
                print(f"Error scraping data: {e}")
            time.sleep(2)
            # Close the new tab and switch back
            driver_jobbkk.close()
            driver_jobbkk.switch_to.window(original_window)
        
        except Exception as e:
            print(f"An error occurred for element with index {x}: {e}")

finally:
    print('Done scraping')
    driver_jobbkk.quit()


{'job_title': 'Data Engineer รับสมัครด่วน', 'company_name': 'บริษัท ซอฟท์นิกซ์ เทคโนโลยี จำกัด', 'industry': 'ประเภทธุรกิจ : คอมพิวเตอร์-ไอที', 'salary': 'เงินเดือน(บาท) : 25,000 - 40,000', 'fulltime': 'รูปแบบงาน : งานประจำ', 'location': 'สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตดินแดง)', 'workday': 'วันทำงาน : จันทร์-ศุกร์', 'dayoff': 'วันหยุด : วันเสาร์, วันอาทิตย์', 'workhour': 'เวลาทำงาน : 09:00 - 18:00', 'responsibility': 'รับผิดชอบงาน Data Integration, Data Pipeline Management\nออกแบบ Data Architecture ติดตั้ง Big Data Platform และเครื่องมือที่เกี่ยวข้อง\nออกแบบโครงสร้างการจัดเก็บและแนวทางการจัดเก็บข้อมูลบน Data Platform\nศึกษาและค้นคว้าเทคโนโลยีที่เหมาะสมเพื่อพัฒนาสินค้าของเราให้มีคุณภาพที่ดีขึ้น และพัฒนาผลิตภัณฑ์ให้เป็นไปตามเป้าหมายของบริษัท', 'gender': 'เพศ : ไม่ระบุ', 'age': 'อายุ(ปี) : 26 - 35', 'education': 'ระดับการศึกษา : ปริญญาตรี ขึ้นไป', 'experience': 'ประสบการณ์(ปี) : 3 - 5', 'job_requirement': 'มีประสบการณ์ด้านการทำงาน Data Warehouse, ETL\nเชี่ยวชาญ SQL, NoSQL และเขียน P

In [10]:
jobbkk_job_data = pd.DataFrame(all_jobs)

In [20]:
jobbkk_job_data


,job_title,company_name,industry,salary,fulltime,location,workday,dayoff,workhour,responsibility,gender,age,education,experience,job_requirement,job_requirement2,welfare_and_benefits,welfare_and_benefits2,company_contact,job_url
0,Data Engineer รับสมัครด่วน,บริษัท ซอฟท์นิกซ์ เทคโนโลยี จำกัด,ประเภทธุรกิจ : คอมพิวเตอร์-ไอที,"เงินเดือน(บาท) : 25,000 - 40,000",รูปแบบงาน : งานประจำ,สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตดินแดง),วันทำงาน : จันทร์-ศุกร์,"วันหยุด : วันเสาร์, วันอาทิตย์",เวลาทำงาน : 09:00 - 18:00,"รับผิดชอบงาน Data Integration, Data Pipeline M...",เพศ : ไม่ระบุ,อายุ(ปี) : 26 - 35,ระดับการศึกษา : ปริญญาตรี ขึ้นไป,ประสบการณ์(ปี) : 3 - 5,"มีประสบการณ์ด้านการทำงาน Data Warehouse, ETL\n...",ยินดีรับนักศึกษาจบใหม่,ทำงานสัปดาห์ละ 5 วัน\nประกันชีวิต\nประกันสุขภา...,สวัสดิการ\n- Birthday Party\n- วันหยุดพักผ่อน ...,ชื่อผู้ติดต่อ : ศิวกรณ์ โนรันต์\nเบอร์ผู้ติดต่...,https://www.jobbkk.com/jobs/detailurgent/18039...
1,Data Engineer รับสมัครด่วน,บริษัท เอพพิค คอนซัลติ้ง จำกัด,ประเภทธุรกิจ : คอมพิวเตอร์-ไอที,วันทำงาน : จันทร์-ศุกร์,ระดับตำแหน่งงาน : เจ้าหน้าที่,"สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตจตุจักร,เ...","วันหยุด : วันเสาร์, วันอาทิตย์",เวลาทำงาน : 08:30 - 17:30,เวลาทำงานอื่นๆ : ไม่ระบุ,"Big Data technology, Hadoop Cloudera Hadoop, C...",เพศ : ไม่ระบุ,อายุ(ปี) : 23 - 35,ระดับการศึกษา : ปริญญาตรี - ปริญญาโท,ประสบการณ์(ปี) : 3 - 10,Excellent with database or data warehouse or E...,ทำงานสัปดาห์ละ 5 วัน\nมีเวลาการทำงานที่ยืดหยุ่...,ชื่อผู้ติดต่อ : สุทัดดาว พงษ์พานิช\nเบอร์ผู้ติ...,"408/48 Phaholyothin Place 11th Fl., แขวงพญาไท ...",ไม่มี\n\nหมอชิต สะพานควาย สนามเป้า อนุสาวรีย์ช...,https://www.jobbkk.com/jobs/detailurgent/18459...
2,Data Engineer รับสมัครด่วน,"R Systems Consulting Services (Thailand) Co., ...",ประเภทธุรกิจ : คอมพิวเตอร์-ไอที,วันทำงาน : ไม่ระบุ,ระดับตำแหน่งงาน : เจ้าหน้าที่,สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(ทุกเขต),"วันหยุด : วันเสาร์, วันอาทิตย์",เวลาทำงาน : 08:30 - 17:30,เวลาทำงานอื่นๆ : ไม่ระบุ,"- Data Engineer, Data Information - Programmin...",เพศ : ไม่ระบุ,อายุ(ปี) : 22 - 55,ระดับการศึกษา : ปริญญาตรี ขึ้นไป,ประสบการณ์(ปี) : 2 - 7,ไม่ระบุ,ทำงานสัปดาห์ละ 5 วัน\nประกันสุขภาพ\nประกันสังค...,ชื่อผู้ติดต่อ : Arpaporn Mekloy\nเบอร์ผู้ติดต่...,2/3 หมู่14 อาคารบางนาทาวเวอร์ เอ ถนนบางนา-ตราด...,ไม่มี\n\nไม่มี\n\nไม่มี\n\nไม่มี\n\nไม่มี,https://www.jobbkk.com/jobs/detailurgent/21546...
3,Data Business รับสมัครด่วน,บริษัท ไวท์ฟร้อนท์ จำกัด,ประเภทธุรกิจ : ธุรกิจอื่นๆ,วันทำงาน : จันทร์-เสาร์,ระดับตำแหน่งงาน : เจ้าหน้าที่,"สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตบางนา,เขต...",วันหยุด : วันอาทิตย์,เวลาทำงาน : 08:30 - 17:30,เวลาทำงานอื่นๆ : ไม่ระบุ,กำหนด Data Sources เพื่อใช้ในการวิเคราะห์ข้อมู...,เพศ : ไม่ระบุ,อายุ(ปี) : 22 ปีขึ้นไป,ระดับการศึกษา : ปริญญาตรี ขึ้นไป,ประสบการณ์(ปี) : 2ปีขึ้นไป,มีทักษะด้านคณิตศาสตร์ และตัวเลข\nเข้าใจเรื่องส...,ยินดีรับนักศึกษาจบใหม่,ค่าทำงานล่วงเวลา\nประกันสังคม\nตามข้อตกลงของบร...,ชื่อผู้ติดต่อ : คุณคิม\nเบอร์ผู้ติดต่อ : 06155...,101/12 อาคาร SPACE101 แขวงบางจาก เขตพระโขนง จั...,https://www.jobbkk.com/jobs/detailurgent/20247...
4,Data Analyst รับสมัครด่วน,บริษัท พีซีซี อินเทอร์เนชันนัล จำกัด,ประเภทธุรกิจ : บริการ,วันทำงาน : ไม่ระบุ,ระดับตำแหน่งงาน : เจ้าหน้าที่,สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตยานนาวา),"วันหยุด : วันเสาร์, วันอาทิตย์",เวลาทำงาน : 08:30 - 17:30,เวลาทำงานอื่นๆ : ไม่ระบุ,1. ดูแลจัดการข้อมูลและรักษา Master Data ให้พร้...,"เพศ : ชาย , หญิง",อายุ(ปี) : 23 ปีขึ้นไป,ระดับการศึกษา : ปริญญาตรี ขึ้นไป,ประสบการณ์(ปี) : 1ปีขึ้นไป,1. วุฒิปริญญาตรี สาขาบัญชี / สาขาวิทยาศาสตร์ข้...,ทำงานสัปดาห์ละ 5 วัน\nประกันสังคม\nตามข้อตกลงข...,ชื่อผู้ติดต่อ : ชุติญา ไสยพร\nเบอร์ผู้ติดต่อ :...,1023 อาคารเอ็มเอส สยาม ชั้น 9 ถ.พระราม 3 แขวงช...,"195,205\n\nไม่มี\n\nไม่มี\n\nไม่มี\n\nไม่มี",https://www.jobbkk.com/jobs/detailurgent/20745...
5,Data Modeler รับสมัครด่วน,บริษัท เอพพิค คอนซัลติ้ง จำกัด,ประเภทธุรกิจ : คอมพิวเตอร์-ไอที,วันทำงาน : จันทร์-ศุกร์,ระดับตำแหน่งงาน : เจ้าหน้าที่,"สถานที่ปฏิบัติงาน : กรุงเทพมหานคร(เขตจตุจักร,เ...","วันหยุด : วันเสาร์, วันอาทิตย์",เวลาทำงาน : 08:30 - 17:30,เวลาทำงานอื่นๆ : ไ